In [1]:
import timeit
from datetime import datetime
import requests
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import urllib
import json

In [2]:
if "auth_cookie" not in locals():
    auth_cookie = None

autorun_execute_count = 0
columns = ['EPOCH', 'NORAD_CAT_ID', 'MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS']

In [3]:
try:
    logs = pd.read_pickle("./logs.pkl.gz","gzip")
except:
    # time, type (trigger, login, skip, fetch_init, fetch_success, process_complete, error), url, epoch, out file
    logs = pd.DataFrame(columns = ["created_on","type","url","epoch","error","output"])
    logs = logs.astype({'created_on': 'datetime64[ns]'})
    
def save_logs():
    logs.to_pickle("./logs.pkl.gz","gzip")
    
def log(log_type,url=None,epoch=None,error=None,output=None):
    global logs
    new_log = {'created_on':np.datetime64('now'), 'type':log_type, 'url':url, 'epoch':epoch, 'error':error, 'output':output}
    logs = logs.append(new_log,ignore_index=True)
    save_logs()

In [25]:
def get_current_file_count():
    return max([int(f[:-7]) for f in listdir('./output/') if f.endswith(".pkl.gz")])

def get_last_epoch():
    df = pd.read_pickle("./output/%05d.pkl.gz" % get_current_file_count(),"gzip")
    return df.EPOCH[-1:].astype("datetime64[ns]").values[0]

def get_space_track_api_count(unit="m"):
    return len(logs[(logs.created_on > (np.datetime64('now') - np.timedelta64(1,unit))) & (logs.type=="fetch_init")])

def space_track_login():
    with open("space-track-credentials.json") as json_file:
        credentials = json.load(json_file)
    print("Logging in space-track.org with identity:",credentials['identity'])
    url = "https://www.space-track.org/ajaxauth/login"
    x = requests.post(url, data = credentials)
    if x.status_code == 200:
        print("Logged in")
        log("login", output="success")
        return x.cookies
    else:
        log("login", error=x.status_code)
        print("Failed with code:",x.status_code)
        return None
    
def generate_url():
    lower = get_last_epoch()
    l = np.datetime_as_string(lower, unit='us')
    if lower.astype('datetime64[Y]').item().year > 2014:
        upper = lower + np.timedelta64(60,'D') # after 2014
    else:
        upper = lower + np.timedelta64(365,'D') # before 2014
    u = np.datetime_as_string(upper, unit='us')
    return (l, u, f'https://www.space-track.org/basicspacedata/query/class/gp_history/ECCENTRICITY/<0.25/MEAN_MOTION//>11.25/EPOCH/>{l},<{u}/OBJECT_TYPE/DEBRIS/orderby/EPOCH asc/limit/100000/format/csv/emptyresult/show')

Before you proceed:
* Create the `space-track-credentials.json` file from template with your own credentials
* Manually test the fetch cell to make sure things are working first
* Set an appropriate intervalTime
* This will probably mess up if your task time is longer than your interval time
* If you don't know what you are doing, ask Tim.

In [5]:
raise Exception('This exception is to stop the cells below from running when you do "Run All Cells"')

Exception: This exception is to stop the cells below from running when you do "Run All Cells"

In [ ]:
%%html
<script>
    var intervalTime = 300000; // 5 minutes in ms
    if (typeof autorun_toggle === 'undefined') {
        var autorun_toggle = null;
    }
    function toggle_autorun() {
        var btn = document.getElementById("autorun_button");
        setTimeout(function(){ // add a delay so the selected cell is correctly set here
            if (autorun_toggle == null) {
                var start_index = IPython.notebook.get_selected_index()+1;
                var end_index = IPython.notebook.get_cells().length;
                console.log("start autorun, start: " + start_index + " end: " + end_index);
                IPython.notebook.execute_cell_range(start_index, end_index);
                //$('div.input').hide(200);
                autorun_toggle = setInterval(function(){
                    console.log("Run cells below");
                    //IPython.notebook.execute_cells_below();
                    IPython.notebook.execute_cell_range(start_index, end_index);
                }, intervalTime);
            } else {
                window.clearInterval(autorun_toggle);
                console.log("clear autorun");
                autorun_toggle = null;
                //$('div.input').show(200);
            }
        }, 100);
    }
</script>
<button id="autorun_button" onclick="toggle_autorun()">Toggle autorun cells below</button>

In [ ]:
start_time = timeit.default_timer()
autorun_execute_count+=1
print(f'Last run local time: {datetime.now()}')
print(f'Total times ran: {autorun_execute_count}')

In [26]:
# Check that we did not exceed Space-Track API limits
# Limit API queries to less than 30 requests per minute / 300 requests per hour

ops_count = 0

while get_space_track_api_count("m") < 25 and get_space_track_api_count("h") < 275:
    ops_count += 1
    # clear expired cookies
    if type(auth_cookie) == requests.cookies.RequestsCookieJar:
        auth_cookie.clear_expired_cookies()
    # log in if needed
    if auth_cookie == None or len(auth_cookie) == 0:
        auth_cookie = space_track_login()
    # stop if can't log in
    if auth_cookie == None:
        log("skip", error = f'Cannot log in')
        break
    # prep url
    start, _, url = generate_url()
    print(f"Fetch at epoch >: {start}")
    log("fetch_init", url=url, epoch=start)
    r = requests.get(url, allow_redirects=True, cookies=auth_cookie)
    fname = f"./tmp/{start}.csv".replace(":","_")
    open(fname, 'wb').write(r.content)
    print(f"Saved CSV: {fname}")
    log("fetch_success", url=url, epoch=start, output=fname)
    oname = ("./output/%05d.pkl.gz" % (get_current_file_count()+1)).replace(":","_")
    pd.read_csv(fname)[columns].to_pickle(oname,"gzip")
    log("process_complete", url=url, epoch=start, output=oname)
    print(f"Processed to: {oname}")
    
if ops_count == 0:
    log("skip", error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')
else:
    log("stop", output=ops_count, error = f'Request limit exceeded: 1m({get_space_track_api_count("m")}) 1h({get_space_track_api_count("h")})')

Fetch at epoch >: 1969-12-31T23:38:16.850400
Saved CSV: ./tmp/1969-12-31T23_38_16.850400.csv
Processed to: ./output/00001.pkl.gz
Fetch at epoch >: 1970-12-30T16:22:02.936352
Saved CSV: ./tmp/1970-12-30T16_22_02.936352.csv
Processed to: ./output/00002.pkl.gz
Fetch at epoch >: 1971-12-29T23:59:25.380384
Saved CSV: ./tmp/1971-12-29T23_59_25.380384.csv
Processed to: ./output/00003.pkl.gz
Fetch at epoch >: 1972-12-27T23:58:34.499423
Saved CSV: ./tmp/1972-12-27T23_58_34.499423.csv
Processed to: ./output/00004.pkl.gz
Fetch at epoch >: 1973-11-28T22:14:50.671103
Saved CSV: ./tmp/1973-11-28T22_14_50.671103.csv
Processed to: ./output/00005.pkl.gz
Fetch at epoch >: 1974-10-10T18:16:41.005344
Saved CSV: ./tmp/1974-10-10T18_16_41.005344.csv
Processed to: ./output/00006.pkl.gz
Fetch at epoch >: 1975-08-15T05:38:05.014464
Saved CSV: ./tmp/1975-08-15T05_38_05.014464.csv
Processed to: ./output/00007.pkl.gz
Fetch at epoch >: 1976-06-10T14:04:54.621119
Saved CSV: ./tmp/1976-06-10T14_04_54.621119.csv
Proc

Processed to: ./output/00064.pkl.gz
Fetch at epoch >: 1989-10-12T16:58:27.581951
Saved CSV: ./tmp/1989-10-12T16_58_27.581951.csv
Processed to: ./output/00065.pkl.gz
Fetch at epoch >: 1989-11-16T07:24:05.410079
Saved CSV: ./tmp/1989-11-16T07_24_05.410079.csv
Processed to: ./output/00066.pkl.gz
Fetch at epoch >: 1989-12-19T15:04:11.682336
Saved CSV: ./tmp/1989-12-19T15_04_11.682336.csv
Processed to: ./output/00067.pkl.gz
Fetch at epoch >: 1990-01-23T21:09:34.373375
Saved CSV: ./tmp/1990-01-23T21_09_34.373375.csv
Processed to: ./output/00068.pkl.gz
Fetch at epoch >: 1990-02-24T14:02:05.186400
Saved CSV: ./tmp/1990-02-24T14_02_05.186400.csv
Processed to: ./output/00069.pkl.gz
Fetch at epoch >: 1990-04-03T09:52:32.024064
Saved CSV: ./tmp/1990-04-03T09_52_32.024064.csv
Processed to: ./output/00070.pkl.gz
Fetch at epoch >: 1990-05-09T06:43:43.657823
Saved CSV: ./tmp/1990-05-09T06_43_43.657823.csv
Processed to: ./output/00071.pkl.gz
Fetch at epoch >: 1990-06-23T23:27:33.778656
Saved CSV: ./tmp

Saved CSV: ./tmp/1995-01-15T21_26_05.012448.csv
Processed to: ./output/00128.pkl.gz
Fetch at epoch >: 1995-02-11T18:30:45.382175
Saved CSV: ./tmp/1995-02-11T18_30_45.382175.csv
Processed to: ./output/00129.pkl.gz
Fetch at epoch >: 1995-03-11T06:26:22.880832
Saved CSV: ./tmp/1995-03-11T06_26_22.880832.csv
Processed to: ./output/00130.pkl.gz
Fetch at epoch >: 1995-04-08T23:54:10.744415
Saved CSV: ./tmp/1995-04-08T23_54_10.744415.csv
Processed to: ./output/00131.pkl.gz
Fetch at epoch >: 1995-05-06T11:07:32.688192
Saved CSV: ./tmp/1995-05-06T11_07_32.688192.csv
Processed to: ./output/00132.pkl.gz
Fetch at epoch >: 1995-06-05T01:50:24.923903
Saved CSV: ./tmp/1995-06-05T01_50_24.923903.csv
Processed to: ./output/00133.pkl.gz
Fetch at epoch >: 1995-06-30T06:49:39.933408
Saved CSV: ./tmp/1995-06-30T06_49_39.933408.csv
Processed to: ./output/00134.pkl.gz
Fetch at epoch >: 1995-07-15T17:28:30.247104
Saved CSV: ./tmp/1995-07-15T17_28_30.247104.csv
Processed to: ./output/00135.pkl.gz
Fetch at epoc

Saved CSV: ./tmp/1999-05-11T02_00_59.883551.csv
Processed to: ./output/00192.pkl.gz
Fetch at epoch >: 1999-06-03T01:07:51.439295
Saved CSV: ./tmp/1999-06-03T01_07_51.439295.csv
Processed to: ./output/00193.pkl.gz
Fetch at epoch >: 1999-06-27T13:17:40.133472
Saved CSV: ./tmp/1999-06-27T13_17_40.133472.csv
Processed to: ./output/00194.pkl.gz
Logging in space-track.org with identity: ttcchen@umich.edu
Logged in
Fetch at epoch >: 1999-07-19T19:45:49.079808
Saved CSV: ./tmp/1999-07-19T19_45_49.079808.csv
Processed to: ./output/00195.pkl.gz
Fetch at epoch >: 1999-08-11T23:27:34.487135
Saved CSV: ./tmp/1999-08-11T23_27_34.487135.csv
Processed to: ./output/00196.pkl.gz
Fetch at epoch >: 1999-09-03T06:23:16.219680
Saved CSV: ./tmp/1999-09-03T06_23_16.219680.csv
Processed to: ./output/00197.pkl.gz
Fetch at epoch >: 1999-09-26T05:22:26.504831
Saved CSV: ./tmp/1999-09-26T05_22_26.504831.csv
Processed to: ./output/00198.pkl.gz
Fetch at epoch >: 1999-10-20T03:33:19.206143
Saved CSV: ./tmp/1999-10-20

/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


...

/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processed to: ./output/00507.pkl.gz
Fetch at epoch >: 2013-11-02T23:26:16.810943
Saved CSV: ./tmp/2013-11-02T23_26_16.810943.csv
Processed to: ./output/00508.pkl.gz
Fetch at epoch >: 2013-11-23T19:08:35.110175
Saved CSV: ./tmp/2013-11-23T19_08_35.110175.csv
Processed to: ./output/00509.pkl.gz
Fetch at epoch >: 2013-12-14T07:36:22.206816
Saved CSV: ./tmp/2013-12-14T07_36_22.206816.csv


/Users/tim/opt/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Processed to: ./output/00510.pkl.gz
Fetch at epoch >: 2014-01-04T12:19:11.858016
Saved CSV: ./tmp/2014-01-04T12_19_11.858016.csv
Processed to: ./output/00511.pkl.gz
Fetch at epoch >: 2014-01-26T03:31:13.461312
Saved CSV: ./tmp/2014-01-26T03_31_13.461312.csv
Processed to: ./output/00512.pkl.gz
Fetch at epoch >: 2014-02-16T12:52:42.773951
Saved CSV: ./tmp/2014-02-16T12_52_42.773951.csv
Processed to: ./output/00513.pkl.gz
Fetch at epoch >: 2014-03-09T11:05:06.866592


KeyboardInterrupt: 

In [ ]:
elapsed = timeit.default_timer() - start_time
print(f"Time used to execute cells: {elapsed} seconds")

In [ ]:
logs

In [24]:
get_last_epoch().astype('datetime64[Y]').item().year

int